In [1]:
%pip install visualkeras

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import numpy as np

2023-10-17 16:27:25.295711: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 16:27:29.204414: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, BatchNormalization 
from keras.preprocessing import image
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
data_path = "/home/ducna/Citrus/data_citrus_level/"
CATEGORIES = ["black_spot", "black_spot_early", "canker", "canker_early", "greening", "greening_early", "melanose", "melanose_early", "healthy"]
IMG_SIZE = 224

#create training data
training = []
def createTrainingData():
    for category in CATEGORIES:
        path = os.path.join(data_path, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img))
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            training.append([new_array, class_num])

createTrainingData()

In [5]:
import random
random.shuffle(training)

In [6]:
X = []
y = []
for features, label in training:
    X.append(features)
    y.append(label)

In [7]:
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

#Normalize X
X = X.astype('float32')
X /= 255

Y = keras.utils.to_categorical(y,9)
print(X.shape)
print(Y.shape)   

(12480, 224, 224, 3)
(12480, 9)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1)
test = X_test
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.22)
print(X_train.shape)

NameError: name 'train_test_split' is not defined

In [14]:
#CNN1
cnn1 = Sequential()
cnn1.add(Conv2D(32, kernel_size = (3,3), padding = "same", activation="relu", input_shape = (224, 224, 3)))
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(BatchNormalization())
cnn1.add(Conv2D(32, kernel_size = (3,3), padding = "same", activation="relu"))
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(Dropout(0.5))
cnn1.add(Dense(256, activation = 'relu'))
cnn1.add(Dense(1, activation='softmax'))
cnn1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 112, 112, 32)      0         
 g2D)                                                            
                                                                 
 batch_normalization_1 (Bat  (None, 112, 112, 32)      128       
 chNormalization)                                                
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 32)      9248      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 56, 56, 32)        0         
 g2D)                                                            
                                                      

In [15]:
cnn1.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])